# Lateral trim

This section describes lateral trim analysis for the example airplane under two scenarios: one-engine inoperative and cross-wind conditions. In these scenarios, the rudder should be able to trim the airplane such that sideslip angle $\beta=0$. In both cases, maximum rudder applied should not exceed $\pm20^{\circ}$. One can also use other requirements (such as spin recovery) for verifying/sizing the rudder. Refer to lecture notes, Section 21.6 in Nicolai and Carichner, and Section 16.4.2 in Raymer for further details.

## One-engine out

The one-engine out scenario mentions that rudder should be able to produce enough yawing moment to hold the aircraft at $\beta=0$ at takeoff speed (1.1 times the stall speed) and CG being at most-aft position. Using yawing moment balance about CG, following condition can be derived (equation 21.24, Nicolai and Carichner) for asymmetric power:

$$
    C_{n_{\delta_r}} \delta_r (\bar{X}_{ac_v} - \bar{X}_{cg}) = \frac{T+D}{qS_wb} \bar{Y}_p,
$$

where $\delta_r$ is the rudder deflection (rad), $T$ is the thrust from one engine, $D$ is the drag from inoperative engine, $q$ is the dynamic pressure, and $\bar{Y}_p$ is the distance between the engine and CG, normalized using wing span $b$. Other terms have been defined earlier. Above equation can be solved for required $\delta_r$ to trim the airplane in one-engine out scenario.

The thrust for operative engine is computed using takeoff power from engine and 80% propeller efficiency, while accounting for installation losses. The drag from the inoperative engine is estimated using $D/q$ for feathered propeller described in [parasitic drag section](../aerodynamics/drag_buildup.ipynb#miscellaneous-drag), along with drag from idle engine. The rudder control derivative can be computed using (equation 21.26, Nicolai)

$$
    C_{n_{\delta_r}} = 0.9 C_{F_{\beta_v}} V_{vt} \tau,
$$

where $C_{F_{\beta_v}}$ is the side-force curve slope for vertical tail (computed earlier), $\tau$ is the rudder effectiveness (obtained from figure 21.14, Nicolai) and $V_{vt}$ is the vertical tail volume coefficient.

Below code block computes thrust and drag for one-engine out scenario:

In [7]:
import numpy as np

rho = 0.00237717 # slugs/cu ft, sea-level
wing_loading = 40 # lbs/sq ft
CLmax_TO = 1.8
Vstall = ( wing_loading * 2 / rho / CLmax_TO )**0.5 # ft/s
V = 1.1 * Vstall # ft/s
q = 0.5 * rho * V**2 # lbs/ft^2

print(f"Speed at takeoff: {V:.2f} ft/s")
print(f"Dynamic pressure at takeoff: {q:.2f} lbs/sq ft")

# Propeller area, sq ft
D = 5.9 # ft
Aprop = np.pi * D**2 / 4

# Thrust
n_prop = 0.8 # prop efficiency
P = 298 # hp, takeoff power
T = 550 * P * n_prop / V # lbs (one-engine only)

D = 0.08 * T + 0.1*Aprop*q # drag of inoperative engine
T = 0.92 * T # account for installation effects

print(f"Thrust from one engine: {T:.2f} lbs")
print(f"Drag from inoperative engine: {D:.2f} lbs")

Speed at takeoff: 150.41 ft/s
Dynamic pressure at takeoff: 26.89 lbs/sq ft
Thrust from one engine: 802.02 lbs
Drag from inoperative engine: 143.25 lbs


Below code block computes required $\delta_r$ for one-engine out scenario:

In [8]:
# Parameters
mac = 4.3 # ft
Xac_v = 0.25 + 17/mac
Xcg = 0.1861
b = 33 # ft
Sw = 134 # sq ft
tau = 0.58 # figure 21.14, Nicolai
V_vt = 0.065
CFbeta_v = 2.7356 # 1/rad
Yp = 6.5/b # distance between engine and CG

Cn_delta_r = 0.9 * CFbeta_v * V_vt * tau # 1/rad

delta_r = (T + D) * Yp / q / Sw / b / Cn_delta_r / ( (Xac_v - Xcg) * mac / b ) * 180 / np.pi

print(f"Required rudder deflection for one-engine out scenario: {delta_r:.1f} degree")

Required rudder deflection for one-engine out scenario: 1.8 degree


Based on the above calculation, the rudder is more than capable of providing directional control during engine out scenario.

## Crosswind

The crosswind scenario requries that aircraft should be able to operate in crosswind speed equal to 20% of takeoff speed. Equivalently, this can be written as $11.5^{\circ}$ of sideslip at takeoff speed. Using moment balance, requried $\delta_r$ can be computed using (equation 21.25, Nicolai and Carichner)

$$
    C_{n_{\beta}} \beta + C_{n_{\delta_r}} \delta_r = 0.
$$

Note that $C_{n_\beta}$ for takeoff conditions is already computed in [yaw static stability section](yaw_derivative.ipynb#final-c-n-beta). Below code block computes required $\delta_r$ for crosswind condition:

In [9]:
Cn_beta = 0.1385 # 1/rad, takeoff
beta = 11.5 * np.pi / 180 # rad, required beta

crosswind_speed = 0.2 * V # ft/s

delta_r = - Cn_beta * beta / Cn_delta_r * 180 / np.pi # deg

print(f"Required rudder deflection for max crosswind speed of {crosswind_speed:.0f} ft/s: {delta_r:.1f} degree")

Required rudder deflection for max crosswind speed of 30 ft/s: -17.2 degree


Based on the above calculation, it can be seen that crosswind condition is the driving requirement for rudder sizing since it requires much larger rudder deflection than one-engine out scenario.

This concludes the lateral trim analysis for the example airplane.